In [39]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [40]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [41]:
from IPython.core.display import HTML
HTML(r"""
<style>
    * {
        font-family: Monaco;
    }
</style>
""")

In [42]:
import os,sys,warnings,re,math,gc,time
warnings.filterwarnings("ignore")
os.cpu_count()

12

In [43]:
os.environ["POLARS_MAX_THREADS"] = "40"
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco",size=15)))
pio.templates.default = "mod+plotly_dark"
from sklearn.metrics import auc,roc_auc_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import miceforest as mf
from miceforest import mean_match_shap,mean_match_default,mean_match_fast_cat
%xmode Minimal

Exception reporting mode: Minimal


In [44]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(40)

polars.config.Config

In [45]:
path_to_train = Path("Downloads/credit_risk/train/")
path_to_test = Path("Downloads/credit_risk/test/")
path_to_features = Path("Downloads/feature_definitions.csv")

In [46]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [47]:
all_train_files = glob("Downloads/credit_risk/train/*.parquet")
all_test_files = glob("Downloads/credit_risk/test/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [48]:
train_files_df.filter(pl.col("filename").str.contains("base"))

index,path,filename
i64,str,str
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""


In [49]:
train_base = (
    pl.scan_parquet(all_train_files[7])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains(["target","date"]),cs.contains("target"))
)
display(train_base.fetch().head())
train_base.collect(streaming=True).shape

case_id_base,month,week,weekday,target
u32,i8,i8,i8,u8
0,1,1,4,0
1,1,1,4,0
2,1,1,5,0
3,1,1,4,0
4,1,1,5,1


(1526659, 5)

In [50]:
test_files_df.filter(pl.col("filename").str.contains("base"))

index,path,filename
i64,str,str
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [51]:
test_base = (
    pl.scan_parquet(all_test_files[33])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains(["date"]),cs.contains("target"))
)
display(test_base.fetch().head())
print(test_base.collect(streaming=True).shape)

case_id_base,month,week,weekday
u32,i8,i8,i8
57543,10,41,2
57549,10,41,2
57551,10,41,2
57552,10,41,3
57569,10,41,2


(10, 4)


In [52]:
train_files_df.filter(pl.col("filename").str.contains("appl"))

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""


## Case_id, num_group1 and num_group2 relationships

<font size=6>

Goal of this competition is to predict risk quality of clients (or to be precise applications of clients), so client and his/her application is entity (case_id) for which your model predict probability of default (credit score).

- depth=0 (what we call static attributes) are attributes that are aggregated on case_id level. Example can be age of client or gender, 1 records per 1 case_id.

- depth=1, those are attributes where we have several records per client/application. Example can be previous applications or loans in credit bureau register, each client can have from 0 to n records. Therefore, for one case_id there might be several records, and to index them we use num_group1

- depth=2: For some attributes with depth=1 we have more detailed information, for example for previous applications we have data about instalments like date of payments or days past due of each payments. It means for each previous application, you can have 0…n records about instalments/payments. And as an index is used num_group2

To sum up, 1 client can have several previous applications, and each of those previous applications can have several records about instalments, payments, days past due, etc.

- num_group1 : 0 -> 19
- num_group2 : 0 -> 11

## Functions

In [53]:
def select_imputable_cols(df:pl.LazyFrame,threshold:int=10) -> pl.LazyFrame:
    col_list = (
        df
        .collect(streaming=True)
        .pipe(lambda df: (df.null_count()/df.shape[0])*100)
        .transpose(include_header=True,column_names=["percentage of nulls"])
        .join(
            feat_df,
            left_on="column",
            right_on="Variable",
            how="left"
        )
        .sort(by="percentage of nulls")
        .select(["column","Description","percentage of nulls"])
        .filter(pl.col("percentage of nulls") < threshold)
        ["column"].to_list()
    )
    _ = df.select(col_list)
    gc.collect()
    return _

In [54]:
def rename_cols(df: pl.LazyFrame) -> pl.LazyFrame:
    return (
        df
        .pipe(
            lambda df: df.rename(
                {_:_ + " " + feat_df.filter(pl.col("Variable") == _).select("Description").item() for _ in df.columns if _ not in ["case_id","num_group1","num_group2"]}
            )
        )
    )

In [55]:
train_past_depth_one = (
    pl.scan_parquet(path_to_train/"train_applprev_1_*.parquet",rechunk=True)
    .drop(cs.contains("date"))
    .select(
        pl.col("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt8),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical),
    )
    .pipe(select_imputable_cols)
    .pipe(rename_cols)
)

In [56]:
train_past_depth_one.fetch().head(10)

case_id,num_group1,cancelreason_3545846M Application cancellation reason.,district_544M District of the address used in the previous loan application.,education_1138M Applicant's education level from their previous application.,postype_4733339M Type of point of sale.,profession_152M Profession of the client during their previous loan application.,rejectreason_755M Reason for previous application rejection.,rejectreasonclient_4145042M Reason for rejection of the client's previous application.,status_219L Previous application status.,actualdpd_943P Days Past Due (DPD) of previous contract (actual).,mainoccupationinc_437A Client's main income amount in their previous application.,credacc_credlmt_575A Credit card credit limit provided for previous applications.,credamount_590A Loan amount or card limit of previous applications.,downpmt_134A Previous application downpayment amount.,credtype_587L Credit type of previous application.,inittransactioncode_279L Type of the initial transaction made in the previous application of the client.,annuity_853A Monthly annuity for previous applications.,pmtnum_8L Number of payments made for the previous application.,tenor_203L Number of instalments in the previous application.
u32,u8,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,cat,cat,f32,f32,f32
2,0,"""a55475b1""","""P136_108_173""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""",0.00,8200.00,0.00,10000.00,0.00,"""CAL""","""CASH""",640.20,24.00,24.00
2,1,"""a55475b1""","""P136_108_173""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""",0.00,8200.00,0.00,16000.00,0.00,"""CAL""","""CASH""",1682.40,12.00,12.00
3,0,"""P94_109_143""","""P131_33_167""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""",0.00,11000.00,0.00,59999.80,0.00,"""CAL""","""CASH""",6140.00,12.00,12.00
4,0,"""P24_27_36""","""P194_82_174""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""",0.00,16000.00,0.00,40000.00,0.00,"""CAL""","""CASH""",2556.60,24.00,24.00
5,0,"""P85_114_140""","""P54_133_26""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""",0.00,62000.00,null,null,null,null,null,null,null,null
6,0,"""P94_109_143""","""P82_154_182""","""a55475b1""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""",0.00,37000.00,0.00,15980.00,0.00,"""CAL""","""CASH""",1773.80,11.00,11.00
6,1,"""P94_109_143""","""P82_154_182""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""",0.00,35000.00,0.00,32000.00,0.00,"""CAL""","""CASH""",4189.60,11.00,11.00
6,2,"""a55475b1""","""a55475b1""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""",0.00,14000.00,0.00,17380.00,0.00,"""COL""","""POS""",1110.40,24.00,24.00
10,0,"""P73_130_169""","""P38_65_49""","""P17_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""",0.00,57000.00,0.00,100000.00,0.00,"""CAL""","""CASH""",10916.60,12.00,12.00


In [57]:
train_past_depth_one_transformed = (
    train_past_depth_one
    .fill_null(strategy="mean")
    .group_by("case_id")
    .agg(
        cs.integer().min(),
        cs.float().max(),
        cs.categorical().mode().get(0)
    )
    .sort(by="case_id")
)
train_past_depth_one_transformed.fetch().head()

case_id,num_group1,actualdpd_943P Days Past Due (DPD) of previous contract (actual).,mainoccupationinc_437A Client's main income amount in their previous application.,credacc_credlmt_575A Credit card credit limit provided for previous applications.,credamount_590A Loan amount or card limit of previous applications.,downpmt_134A Previous application downpayment amount.,annuity_853A Monthly annuity for previous applications.,pmtnum_8L Number of payments made for the previous application.,tenor_203L Number of instalments in the previous application.,cancelreason_3545846M Application cancellation reason.,district_544M District of the address used in the previous loan application.,education_1138M Applicant's education level from their previous application.,postype_4733339M Type of point of sale.,profession_152M Profession of the client during their previous loan application.,rejectreason_755M Reason for previous application rejection.,rejectreasonclient_4145042M Reason for rejection of the client's previous application.,status_219L Previous application status.,credtype_587L Credit type of previous application.,inittransactioncode_279L Type of the initial transaction made in the previous application of the client.
u32,u8,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
2,0,0.00,8200.00,0.00,16000.00,0.00,1682.40,24.00,24.00,"""a55475b1""","""P136_108_173""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""","""CAL""","""CASH"""
3,0,0.00,11000.00,0.00,59999.80,0.00,6140.00,12.00,12.00,"""P94_109_143""","""P131_33_167""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""CAL""","""CASH"""
4,0,0.00,16000.00,0.00,40000.00,0.00,2556.60,24.00,24.00,"""P24_27_36""","""P194_82_174""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""","""CAL""","""CASH"""
5,0,0.00,62000.00,2161.10,45335.69,471.80,3745.33,18.44,18.44,"""P85_114_140""","""P54_133_26""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""","""CAL""","""CASH"""
6,0,0.00,37000.00,0.00,32000.00,0.00,4189.60,24.00,24.00,"""P94_109_143""","""P82_154_182""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""CAL""","""CASH"""


In [58]:
train_files_df.filter(pl.col("filename").str.contains("applprev_2"))

index,path,filename
i64,str,str
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""


In [59]:
train_past_depth_two = (
    pl.scan_parquet(all_train_files[0])
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains(["num_group"]).cast(pl.UInt8),
        cs.string().cast(pl.Categorical)
    )
    .pipe(select_imputable_cols)
    .pipe(rename_cols)
)
train_past_depth_two.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M Card blocking reason.
u32,u8,u8,cat
2,1,1,null
2,0,1,null
2,0,0,null
2,1,0,null
3,0,1,null


In [61]:
train_past_depth_two_transformed = (
    train_past_depth_two
    .with_columns(
        cs.contains("Card").fill_null(value=cs.contains("Card").mode().first())
    )
    .filter((pl.col("num_group1") == 0) & (pl.col("num_group2") == 0))
)
train_past_depth_two_transformed.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M Card blocking reason.
u32,u8,u8,cat
2,0,0,null
3,0,0,null
4,0,0,null
5,0,0,null
6,0,0,null


## Static Files

In [62]:
train_files_df.filter(cs.contains("file").str.contains("train_static_0_"))

index,path,filename
i64,str,str
21,"""Downloads/credit_risk/train/train_static_0_0.parquet""","""train_static_0_0.parquet"""
16,"""Downloads/credit_risk/train/train_static_0_1.parquet""","""train_static_0_1.parquet"""


In [63]:
train_static_base = (
    pl.scan_parquet("Downloads/credit_risk/train/train_static_0_*.parquet")
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
)
train_static_base.fetch().head()

case_id,annuity_780A Monthly annuity amount.,applicationcnt_361L Number of applications associated with the same email address as the client.,applications30d_658L Number of applications made by the client in the last 30 days.,applicationscnt_1086L Number of applications associated with the same phone number.,applicationscnt_464L Number of applications made in the last 30 days by other clients with the same employer as the applicant.,applicationscnt_629L Number of applications with the same employer in the last 7 days.,applicationscnt_867L Number of applications associated with the same mobile phone.,clientscnt12m_3712952L Number of clients that have used the same mobile phone as the applicant in the past 12 months.,clientscnt3m_3712950L Number of clients who have the same mobile phone number in the last 3 months.,clientscnt6m_3712949L Total number of clients who have used the same mobile number in the last 6 months.,clientscnt_100L Number of applications with matching employer's phone and client's.,clientscnt_1022L Number of clients sharing the same mobile phone.,clientscnt_1071L Number of applications where the alternative phone number matches that of the client.,clientscnt_1130L Number of applications where client's phone number matches the alternative phone contact.,clientscnt_157L Number of clients whose employer has the same phone number as the client.,clientscnt_257L Number of clients that share an alternative phone number with the applicant.,clientscnt_304L Number of clients with the same phone number.,clientscnt_360L Number of clients that have the same alternative phone number and employer's phone number.,clientscnt_493L Number of clients with matching phone numbers for both the employer and the client.,clientscnt_533L Number of clients with same client's and alternative's phone number,clientscnt_887L Number of clients sharing the same employer's phone number.,clientscnt_946L Number of clients with matching mobile and employer's number.,credamount_770A Loan amount or credit card limit.,deferredmnthsnum_166L Number of deferred months.,disbursedcredamount_1113A Disbursed credit amount after consolidation.,downpmt_116A Amount of downpayment.,homephncnt_628L Number of distinct home phones on client's application.,mobilephncnt_593L Number of persons with the same mobile phone number.,numactivecreds_622L Number of active credits.,numactivecredschannel_414L Number of active credits.,numactiverelcontr_750L Number of active revolving credits.,numcontrs3months_479L Number of contracts in last 3 months.,numnotactivated_1143L Number of non-activated credits.,numpmtchanneldd_318L Number of previous loan contracts for the applicant that had direct debit as payment channel.,numrejects9m_859L Number of credit applications that were rejected in the last 9 months.,sellerplacecnt_915L Number of sellerplaces where the same client's document was used.,sellerplacescnt_216L Number of sellerplaces where the same client's mobile phone was used.,lastapprcommoditycat_1041M Commodity category of the last loan applications made by the applicant.,lastapprcommoditytypec_5251766M Commodity type of the last application.,lastcancelreason_561M Cancellation reason of the last application.,lastrejectcommoditycat_161M Category of commodity in the applicant's last rejected application.,lastrejectcommodtypec_5251769M Commodity type of the last rejected application.,lastrejectreason_759M Reason for rejection on the most recent rejected application.,lastrejectreasonclient_4145040M Reason for the client's last loan rejection.,previouscontdistrict_112M Contact district of the client's previous approved application.,credtype_322L Type of credit.,inittransactioncode_186L Transaction type of the initial credit transaction.,totaldebt_9A Total amount of debt.,annuitynextmonth_57A Next month's amount of annuity.,currdebt_22A Current debt amount of the client.,currdebtcredtyperange_828A Current amount of debt of the applicant.,numinstls_657L Number of instalments.,tot

In [64]:
train_static_base_transformed = (
    train_static_base
    .with_columns(
        cs.categorical().fill_null(value=cs.categorical().mode().first()),
        cs.float().fill_null(strategy="mean")
    )
)
train_static_base_transformed.fetch().head()

case_id,annuity_780A Monthly annuity amount.,applicationcnt_361L Number of applications associated with the same email address as the client.,applications30d_658L Number of applications made by the client in the last 30 days.,applicationscnt_1086L Number of applications associated with the same phone number.,applicationscnt_464L Number of applications made in the last 30 days by other clients with the same employer as the applicant.,applicationscnt_629L Number of applications with the same employer in the last 7 days.,applicationscnt_867L Number of applications associated with the same mobile phone.,clientscnt12m_3712952L Number of clients that have used the same mobile phone as the applicant in the past 12 months.,clientscnt3m_3712950L Number of clients who have the same mobile phone number in the last 3 months.,clientscnt6m_3712949L Total number of clients who have used the same mobile number in the last 6 months.,clientscnt_100L Number of applications with matching employer's phone and client's.,clientscnt_1022L Number of clients sharing the same mobile phone.,clientscnt_1071L Number of applications where the alternative phone number matches that of the client.,clientscnt_1130L Number of applications where client's phone number matches the alternative phone contact.,clientscnt_157L Number of clients whose employer has the same phone number as the client.,clientscnt_257L Number of clients that share an alternative phone number with the applicant.,clientscnt_304L Number of clients with the same phone number.,clientscnt_360L Number of clients that have the same alternative phone number and employer's phone number.,clientscnt_493L Number of clients with matching phone numbers for both the employer and the client.,clientscnt_533L Number of clients with same client's and alternative's phone number,clientscnt_887L Number of clients sharing the same employer's phone number.,clientscnt_946L Number of clients with matching mobile and employer's number.,credamount_770A Loan amount or credit card limit.,deferredmnthsnum_166L Number of deferred months.,disbursedcredamount_1113A Disbursed credit amount after consolidation.,downpmt_116A Amount of downpayment.,homephncnt_628L Number of distinct home phones on client's application.,mobilephncnt_593L Number of persons with the same mobile phone number.,numactivecreds_622L Number of active credits.,numactivecredschannel_414L Number of active credits.,numactiverelcontr_750L Number of active revolving credits.,numcontrs3months_479L Number of contracts in last 3 months.,numnotactivated_1143L Number of non-activated credits.,numpmtchanneldd_318L Number of previous loan contracts for the applicant that had direct debit as payment channel.,numrejects9m_859L Number of credit applications that were rejected in the last 9 months.,sellerplacecnt_915L Number of sellerplaces where the same client's document was used.,sellerplacescnt_216L Number of sellerplaces where the same client's mobile phone was used.,lastapprcommoditycat_1041M Commodity category of the last loan applications made by the applicant.,lastapprcommoditytypec_5251766M Commodity type of the last application.,lastcancelreason_561M Cancellation reason of the last application.,lastrejectcommoditycat_161M Category of commodity in the applicant's last rejected application.,lastrejectcommodtypec_5251769M Commodity type of the last rejected application.,lastrejectreason_759M Reason for rejection on the most recent rejected application.,lastrejectreasonclient_4145040M Reason for the client's last loan rejection.,previouscontdistrict_112M Contact district of the client's previous approved application.,credtype_322L Type of credit.,inittransactioncode_186L Transaction type of the initial credit transaction.,totaldebt_9A Total amount of debt.,annuitynextmonth_57A Next month's amount of annuity.,currdebt_22A Current debt amount of the client.,currdebtcredtyperange_828A Current amount of debt of the applicant.,numinstls_657L Number of instalments.,tot

In [65]:
train_files_df.filter(cs.contains("file").str.contains("static"))

index,path,filename
i64,str,str
21,"""Downloads/credit_risk/train/train_static_0_0.parquet""","""train_static_0_0.parquet"""
16,"""Downloads/credit_risk/train/train_static_0_1.parquet""","""train_static_0_1.parquet"""
3,"""Downloads/credit_risk/train/train_static_cb_0.parquet""","""train_static_cb_0.parquet"""


In [95]:
train_static_external =(
    pl.scan_parquet("Downloads/credit_risk/train/train_static_cb_0.parquet")
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
)
train_static_external.fetch().head()

case_id,description_5085714M Categorization of clients by credit bureau.,education_1103M Level of education of the client provided by external source.,education_88M Education level of the client.,maritalst_385M Marital status of the client.,maritalst_893M Marital status of the client,days120_123L Number of credit bureau queries for the last 120 days.,days180_256L Number of credit bureau queries for last 180 days.,days30_165L Number of credit bureau queries for the last 30 days.,days360_512L Number of Credit Bureau queries for last 360 days.,days90_310L Number of credit bureau queries for the last 90 days.,firstquarter_103L Number of results obtained from credit bureau in the first quarter.,fourthquarter_440L Number of results in fourth quarter.,numberofqueries_373L Number of queries to credit bureau.,secondquarter_766L Number of results in second quarter.,thirdquarter_1082L Number of results in third quarter.,dateofbirth_337D Client's date of birth.
u32,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat
357,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,null
381,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,null
388,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",6.00,8.00,2.00,10.00,4.00,2.00,6.00,10.00,3.00,5.00,"""1989-04-01"""
405,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",0.00,0.00,0.00,1.00,0.00,0.00,4.00,1.00,2.00,0.00,"""1974-03-01"""
409,"""a55475b1""","""717ddd49""","""a55475b1""","""a7fcb6e5""","""a55475b1""",2.00,3.00,0.00,3.00,1.00,4.00,1.00,3.00,0.00,4.00,"""1993-06-01"""


In [90]:
train_static_external_transformed = (
    train_static_external
    .drop(cs.contains("birth"))
    .with_columns(
        cs.float().fill_null(strategy="mean")
    )
)
train_static_external_transformed.fetch().head()

case_id,description_5085714M Categorization of clients by credit bureau.,education_1103M Level of education of the client provided by external source.,education_88M Education level of the client.,maritalst_385M Marital status of the client.,maritalst_893M Marital status of the client,days120_123L Number of credit bureau queries for the last 120 days.,days180_256L Number of credit bureau queries for last 180 days.,days30_165L Number of credit bureau queries for the last 30 days.,days360_512L Number of Credit Bureau queries for last 360 days.,days90_310L Number of credit bureau queries for the last 90 days.,firstquarter_103L Number of results obtained from credit bureau in the first quarter.,fourthquarter_440L Number of results in fourth quarter.,numberofqueries_373L Number of queries to credit bureau.,secondquarter_766L Number of results in second quarter.,thirdquarter_1082L Number of results in third quarter.
u32,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
357,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2.10,2.88,0.69,4.88,1.62,1.79,2.40,4.88,1.73,2.25
381,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2.10,2.88,0.69,4.88,1.62,1.79,2.40,4.88,1.73,2.25
388,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",6.00,8.00,2.00,10.00,4.00,2.00,6.00,10.00,3.00,5.00
405,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",0.00,0.00,0.00,1.00,0.00,0.00,4.00,1.00,2.00,0.00
409,"""a55475b1""","""717ddd49""","""a55475b1""","""a7fcb6e5""","""a55475b1""",2.00,3.00,0.00,3.00,1.00,4.00,1.00,3.00,0.00,4.00


## Train Person

In [113]:
train_files_df.filter(pl.col("filename").str.contains("person"))

index,path,filename
i64,str,str
5,"""Downloads/credit_risk/train/train_person_1.parquet""","""train_person_1.parquet"""
30,"""Downloads/credit_risk/train/train_person_2.parquet""","""train_person_2.parquet"""


In [147]:
train_person_depth_one = (
    pl.scan_parquet(all_train_files[5])
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.by_name("num_group1").cast(pl.UInt8),
        cs.boolean(),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical()),
    )
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
)
train_person_depth_one.fetch().head()

case_id,num_group1,contaddr_district_15M Zip code of a contact person's address.,contaddr_zipcode_807M Zip code of contact address.,education_927M Education level of the person.,empladdr_district_926M District where the employer's address is located.,empladdr_zipcode_114M Zipcode of employer's address.,language1_981M The primary language of the person.,registaddr_district_1083M District of person's registered address.,registaddr_zipcode_184M Registered address's zip code of a person.,persontype_1072L Person type.,role_1084L Type of contact role.,type_25L Contact type of a person.
u32,u8,cat,cat,cat,cat,cat,cat,cat,cat,f32,cat,cat
0,0,"""P88_18_84""","""P167_100_165""","""P97_36_170""","""P142_57_166""","""P167_100_165""","""P10_39_147""","""P88_18_84""","""P167_100_165""",1.00,"""CL""","""PRIMARY_MOBILE"""
0,1,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",1.00,"""EM""","""PHONE"""
0,2,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",4.00,"""PE""","""PHONE"""
0,3,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",5.00,"""PE""","""PHONE"""
1,0,"""P103_93_94""","""P176_37_166""","""P97_36_170""","""P49_46_174""","""P160_59_140""","""P10_39_147""","""P103_93_94""","""P176_37_166""",1.00,"""CL""","""PRIMARY_MOBILE"""


In [151]:
train_person_depth_one_transformed = (
    train_person_depth_one
    .with_columns(
        cs.categorical().fill_null(value=cs.categorical().mode().first()),
        cs.float().fill_null(strategy="mean")
    )
)
train_person_depth_one_transformed.fetch().head()

case_id,num_group1,contaddr_district_15M Zip code of a contact person's address.,contaddr_zipcode_807M Zip code of contact address.,education_927M Education level of the person.,empladdr_district_926M District where the employer's address is located.,empladdr_zipcode_114M Zipcode of employer's address.,language1_981M The primary language of the person.,registaddr_district_1083M District of person's registered address.,registaddr_zipcode_184M Registered address's zip code of a person.,persontype_1072L Person type.,role_1084L Type of contact role.,type_25L Contact type of a person.
u32,u8,cat,cat,cat,cat,cat,cat,cat,cat,f32,cat,cat
0,0,"""P88_18_84""","""P167_100_165""","""P97_36_170""","""P142_57_166""","""P167_100_165""","""P10_39_147""","""P88_18_84""","""P167_100_165""",1.00,"""CL""","""PRIMARY_MOBILE"""
0,1,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",1.00,"""EM""","""PHONE"""
0,2,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",4.00,"""PE""","""PHONE"""
0,3,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",5.00,"""PE""","""PHONE"""
1,0,"""P103_93_94""","""P176_37_166""","""P97_36_170""","""P49_46_174""","""P160_59_140""","""P10_39_147""","""P103_93_94""","""P176_37_166""",1.00,"""CL""","""PRIMARY_MOBILE"""


In [157]:
(
    pl.scan_parquet(all_train_files[30])
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt8),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)

<LazyFrame [11 cols, {"case_id": UInt32 … "relatedpersons_role_762T": Categorical(ordering='physical')}] at 0x7FE029E67DD0>